In [1]:
from os.path import join as pathjoin
from data_processing import *
from interpretation import *
from models import *
from training import *

Let's download all the data.

In [2]:
DATA_DIR = '/home/mlepekhin/data'
MODELS_DIR = '/home/mlepekhin/models'
MODEL_ID = 'allennlp_simple_cnn_en'
!mkdir {pathjoin(MODELS_DIR, MODEL_ID)}
CHECKPOINTS_DIR = pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints')
BEST_MODEL = pathjoin(CHECKPOINTS_DIR, 'best.th')

mkdir: cannot create directory ‘/home/mlepekhin/models/allennlp_simple_cnn_en’: File exists


In [8]:
train_dataset_reader = build_dataset_reader(None, lower=True)
val_dataset_reader = build_dataset_reader(None, lower=True)

train_data, dev_data = read_data(
    pathjoin(DATA_DIR, "en_train"), 
    pathjoin(DATA_DIR, "en_test"),
    train_dataset_reader, 
    val_dataset_reader
)

vocab = build_vocab(train_data + dev_data)

train_data.index_with(vocab)
dev_data.index_with(vocab)

Reading data
<class 'data_processing.ClassificationDatasetReader'> /home/mlepekhin/data/en_train



Building the vocabulary


In [9]:
model = build_simple_cnn_model(
    vocab, emb_size=256, output_dim=128, num_filters=32, ngram_filter_sizes=(2, 3, 4, 5)
)

Building the model


In [10]:
if torch.cuda.is_available():
    cuda_device = 0
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
print(cuda_device)

0


In [11]:
!rm -rf {CHECKPOINTS_DIR}
!mkdir -p {CHECKPOINTS_DIR}

In [12]:
train_loader, dev_loader = build_data_loaders(train_data, dev_data)

# You obviously won't want to create a temporary file for your training
# results, but for execution in binder for this course, we need to do this.

trainer = build_classifier_trainer(
    model,
    pathjoin(MODELS_DIR, MODEL_ID, 'checkpoints'),
    train_loader,
    dev_loader,
    10,
    cuda_device=cuda_device,
    learning_rate=0.001,
)
print("Starting training")
trainer.train()
print("Finished training")

You provided a validation dataset but patience was set to None, meaning that early stopping is disabled


Starting training



Finished training


In [13]:
!rm "{CHECKPOINTS_DIR}"/*.json "{CHECKPOINTS_DIR}"/model* "{CHECKPOINTS_DIR}"/training*
trainer.model.vocab.save_to_files(pathjoin(MODELS_DIR, MODEL_ID, 'vocab'))
!ls -lh "{CHECKPOINTS_DIR}"

total 83M
-rw-rw-r-- 1 mlepekhin mlepekhin  83M сен 23 17:39 best.th
drwxrwxr-x 4 mlepekhin mlepekhin 4,0K сен 23 17:39 log
